In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
df = pd.read_csv("data/telco.csv", index_col="customerID")
df.shape

(7043, 20)

In [3]:
# TotalCharges 가 수치 타입이 아니기 때문에 수치 연산을 위해 숫자 형태로 변경합니다.
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

In [4]:
df.head(1)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No


In [5]:
df = df.dropna()
df.isnull().sum().sum()

0

In [6]:
label_name = "Churn"

In [7]:
df_ohe = pd.get_dummies(df.drop(label_name, axis=1))
df_ohe[label_name] = df[label_name] == "Yes"
df_ohe.shape

(7032, 46)

In [8]:
df_ohe[label_name].value_counts()

False    5163
True     1869
Name: Churn, dtype: int64

## data split

In [9]:
from sklearn.model_selection import train_test_split

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    df_ohe.drop(label_name, axis=1), df_ohe[label_name], test_size=0.2, random_state=42)

X_train_raw.shape, X_test_raw.shape, y_train_raw.shape, y_test_raw.shape

((5625, 45), (1407, 45), (5625,), (1407,))

In [10]:
# Tensor 변환
X_train = torch.Tensor(X_train_raw.values)
X_valid = torch.Tensor(X_test_raw.values)

X_train.shape

torch.Size([5625, 45])

In [11]:
# Label
y_train = torch.Tensor(y_train_raw.values)
y_valid = torch.Tensor(y_test_raw.values)
print(y_train.shape, y_valid.shape)
y_train[:5]

torch.Size([5625]) torch.Size([1407])


tensor([1., 1., 1., 0., 0.])

In [12]:
y_train.shape

torch.Size([5625])

## model

In [13]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.seq(x)
        return x

# 모델 초기화
input_size = X_train.shape[1]
model = LogisticRegression(input_size)

# 손실 함수 및 optimizer 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

optimizer에 등록된 모든 매개변수의 gradient를 0으로 초기화하는 메서드입니다. 이를 호출하지 않으면, backward() 함수 호출 시 이전에 계산된 gradient 값과 현재 gradient 값이 누적되어 학습이 제대로 이루어지지 않을 수 있습니다.

따라서 모델의 학습을 시작하기 전에, optimizer.zero_grad()를 호출하여 gradient 값을 초기화해야 합니다. 예를 들어, 다음과 같은 코드에서는 각 학습 루프(iteration)마다 optimizer.zero_grad()를 호출하여 gradient를 초기화합니다.

## train

In [14]:
# 모델 초기화
input_size = X_train.shape[1]
model = LogisticRegression(input_size)

# 손실 함수 및 optimizer 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
num_epochs = 1000
best_val_loss = 0.0
num_bad_epochs = 0
early_stop_patience = 50
for epoch in range(num_epochs):
    # forward + backward + optimize
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)
    loss.backward()
    optimizer.step()

    # 중간 결과 출력
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    # 검증 데이터에 대한 성능 측정
    with torch.no_grad():
        model.eval()
        val_outputs = model(X_valid)
        val_loss = criterion(val_outputs.squeeze(), y_valid)

        # 검증 데이터에 대한 정확도 계산
        val_preds = (val_outputs > 0.5).float()
        val_acc = (val_preds == y_valid).float().mean()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss.item():.4f}, Val Acc: {val_acc.item():.4f}')

        # 검증 데이터에 대한 Loss 가 early_stop_patience번 연속 개선되지 않으면 조기 종료
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            num_bad_epochs = 0
            torch.save(model.state_dict(), 'best_model.pt')
        else:
            num_bad_epochs += 1
            if num_bad_epochs == early_stop_patience:
                print("Early stopping")
                break

Epoch [10/1000], Val Loss: 0.6940, Val Acc: 0.7342
Epoch [20/1000], Val Loss: 0.6936, Val Acc: 0.7342
Epoch [30/1000], Val Loss: 0.6934, Val Acc: 0.7342
Epoch [40/1000], Val Loss: 0.6933, Val Acc: 0.7342
Epoch [50/1000], Val Loss: 0.6932, Val Acc: 0.7342
Early stopping



모델이 nan을 반환한다면

* 학습률이 너무 크거나 작음
* Gradient가 폭주하거나 소실
* 데이터가 너무 불균형하거나 잘못된 처리가 있을 수 있음

다음의 방법을 시도해 볼 수 있음

* Gradient Clipping: gradient 값을 일정 범위로 제한하여 너무 큰 gradient로 인해 발생하는 문제를 해결할 수 있습니다.
* Weight Initialization: 모델 파라미터의 초기값을 조정하여 학습이 잘 이루어지도록 돕는 방법입니다.
* Learning Rate Scheduler: 학습이 진행됨에 따라 learning rate를 감소시키는 방법입니다. 처음에는 큰 learning rate로 학습을 시작하다가, 학습이 진행될수록 learning rate를 작게 조정하면, 모델의 성능을 높일 수 있습니다.
* Regularization: 모델의 overfitting을 막는 방법으로, L1/L2 regularization, dropout, batch normalization 등이 있습니다.

## Validation

* torch.no_grad()는 PyTorch에서 gradient 계산을 수행하지 않도록 하는 context manager입니다. 이를 사용하면 모델의 inference 과정에서 gradient 계산을 하지 않아 메모리 사용량을 줄일 수 있습니다. 또한 gradient 계산이 필요하지 않은 validation, test 데이터셋 등에서 사용하여 불필요한 계산을 방지할 수 있습니다.


In [18]:
with torch.no_grad():
    outputs = model(X_valid)
    y_valid_predict = (outputs >= 0.5).float()
    y_valid_predict = y_valid_predict.squeeze()
    
(y_valid.squeeze() == y_valid_predict).numpy().mean()

0.7341862117981521